In [1]:
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from pathlib import Path

import torch.optim as optim
import pandas as pd
import torch.nn as nn
import torch

from data_loader import BraTSDataset
from unet_simple import Unet

# Unet modification

1. Sum as aggregator
2. Upsample in `forward`

https://arxiv.org/abs/1505.04597

<!-- ![title](https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/u-net-architecture.png) -->

In [2]:
data_folder = Path('/home/anvar/work/data/brats_slices/')
df = pd.read_csv(data_folder / 'meta.csv', index_col=0)
dataset = BraTSDataset(df, data_folder)

dataset_loader = torch.utils.data.DataLoader(dataset,
                                             batch_size=4, shuffle=True,
                                             num_workers=8)

In [3]:
device = ("cuda" if torch.cuda.is_available() else 'cpu')
model = Unet().to(device)

criterion = nn.BCEWithLogitsLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [4]:
for epoch in range(5):
    epoch_loss = 0
    for X_batch, y_batch in tqdm(dataset_loader):
        
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        y_pred = model(X_batch.float())
        
        loss = criterion(y_pred, y_batch.float().unsqueeze(1))        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()        

    print(f'Epoch {epoch+0:03}: | Loss: {epoch_loss/len(dataset_loader):.5f}')

/home/anvar/anaconda3/lib/python3.8/site-packages/torch/tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/home/anvar/anaconda3/lib/python3.8/site-packages/torch/tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/home/anvar/anaconda3/lib/python3.8/site-packages/torch/tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/home/anvar/anaconda3/lib/python3.8/site-packages/torch/tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/home/anvar/anaconda3/lib/python3.8/site-packages/torch/tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/home/anvar/anaconda3/lib/python3.8/site-packages/torch/tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is

KeyboardInterrupt: 